In [11]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = 'From Monday to Friday most people are busy working or studying, '\
       'but in the evenings and weekends they are free and _ themselves.'

tokenized_text = tokenizer.tokenize(text)

print(tokenized_text)

['from', 'monday', 'to', 'friday', 'most', 'people', 'are', 'busy', 'working', 'or', 'studying', ',', 'but', 'in', 'the', 'evenings', 'and', 'weekends', 'they', 'are', 'free', 'and', '_', 'themselves', '.']


In [12]:
masked_index = tokenized_text.index('_')

tokenized_text[masked_index] = '[MASK]'

print(tokenized_text)

['from', 'monday', 'to', 'friday', 'most', 'people', 'are', 'busy', 'working', 'or', 'studying', ',', 'but', 'in', 'the', 'evenings', 'and', 'weekends', 'they', 'are', 'free', 'and', '[MASK]', 'themselves', '.']


In [14]:
candidates = ['love', 'work', 'enjoy', 'play']

candidates_ids = tokenizer.convert_tokens_to_ids(candidates)

print(candidates_ids)

[2293, 2147, 5959, 2377]


In [16]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

print(indexed_tokens)

[2013, 6928, 2000, 5958, 2087, 2111, 2024, 5697, 2551, 2030, 5702, 1010, 2021, 1999, 1996, 16241, 1998, 13499, 2027, 2024, 2489, 1998, 103, 3209, 1012]


In [18]:
segments_ids = [0] * len(tokenized_text)

tokens_tensor = torch.tensor([indexed_tokens])

segments_tensors = torch.tensor([segments_ids])

print('Tokens_tensor: ', tokens_tensor)

print('Segments_tensor: ', segments_tensors)



Tokens_tensor:  tensor([[ 2013,  6928,  2000,  5958,  2087,  2111,  2024,  5697,  2551,  2030,
          5702,  1010,  2021,  1999,  1996, 16241,  1998, 13499,  2027,  2024,
          2489,  1998,   103,  3209,  1012]])
Segments_tensor:  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]])


In [23]:
language_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

language_model.eval()

predictions = language_model(tokens_tensor, segments_tensors)

print(predictions)

print(predictions.size)

tensor([[[ -7.3524,  -7.3705,  -7.3813,  ...,  -6.7036,  -6.6253,  -4.1512],
         [ -8.5638,  -9.0042,  -9.0043,  ...,  -6.2188,  -5.9293,  -5.9213],
         [-16.1689, -15.9357, -16.1867,  ..., -12.8809, -13.8796,  -4.9378],
         ...,
         [ -5.7302,  -6.0601,  -5.5552,  ...,  -5.6121,  -4.4892,  -7.0992],
         [ -8.7442,  -8.8843,  -8.7663,  ...,  -8.4347,  -7.4472,  -6.5295],
         [-10.8952, -10.9725, -10.8689,  ..., -10.7548, -10.0049,  -7.0908]]],
       grad_fn=<AddBackward0>)
<built-in method size of Tensor object at 0x00000215DA8E3BD0>


In [21]:
predictions_candidates = predictions[0, masked_index, candidates_ids]

print(predictions_candidates)

tensor([1.7453, 3.2638, 6.1241, 2.6489], grad_fn=<IndexBackward>)


In [22]:
answer_idx = torch.argmax(predictions_candidates).item()

print(f'The most likely word is "{candidates[answer_idx]}".')

The most likely word is "enjoy".
